In [1]:
import tensorflow as tf
import random
import os
import pickle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/캡스톤/data3/'

In [5]:
with open(path+"train_data.pickle","rb") as fr:
    train_data = pickle.load(fr)
    
with open(path+"valid_data.pickle","rb") as fr:
    valid_data = pickle.load(fr)
    
with open(path+"test_data.pickle","rb") as fr:
    test_data = pickle.load(fr)

In [6]:
with open(path+"train_target.pickle","rb") as fr:
    train_target = pickle.load(fr)
    
with open(path+"valid_target.pickle","rb") as fr:
    valid_target = pickle.load(fr)
    
with open(path+"test_target.pickle","rb") as fr:
    test_target = pickle.load(fr)

In [ ]:
test_data[0]

array([[[ 4399,  1651,  2818, ...,     0,     0,     0],
        [ 5458,  1439,  2818, ...,     0,     0,     0],
        [ 1587,  9017,  1439, ...,     0,     0,     0],
        [ 9038,  2818,   574, ...,     0,     0,     0]],

       [[ 4399,  1651,  2818, ...,     0,     0,     0],
        [ 5458,  1439,  2818, ...,     0,     0,     0],
        [ 1587,  9017,  1439, ...,     0,     0,     0],
        [ 9038,  2818,   574, ...,     0,     0,     0]],

       [[ 4399,  1651,  2818, ...,     0,     0,     0],
        [ 5458,  1439,  2818, ...,     0,     0,     0],
        [ 1587,  9017,  1439, ...,     0,     0,     0],
        [ 9038,  2818,   574, ...,     0,     0,     0]],

       ...,

       [[ 1651,   894,  9017, ...,     0,     0,     0],
        [ 4723, 15169,  4723, ...,     0,     0,     0],
        [ 5385,  2818, 11916, ...,     0,     0,     0],
        [ 4723,  4723,  5385, ...,     0,     0,     0]],

       [[ 1651,   894,  9017, ...,     0,     0,     0],
        [ 

## Model

In [7]:
from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Multiply, Bidirectional, Flatten, BatchNormalization, Add
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPool1D
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
import numpy as np
import keras.backend as K
from random import randint

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
description_num = 3
#current_path ='data/'
#local_path = 'data/'

In [8]:
# set parameters:
# V = 142381
# V = 226928
V = 31466  # 6158 # 33531 # len(tokenizer.word_index)
embedding_dim = 50
max_len = 1253   # 77   # 1945  # maxlen
filters_num = 64
kernel_size = 3

In [9]:
# description_input=Input(shape=(10,50))
# description_validation=Input(shape=(50,))

input_1 = Input(shape=(description_num, max_len))
input_2 = Input(shape=(max_len,))
embedding = Embedding(input_dim=V,
                      output_dim=embedding_dim,
                      input_length=max_len)
conv1d = Conv1D(filters=filters_num,
                kernel_size=kernel_size,
                activation='relu',
             #    padding='same'
                )
maxpool1d = MaxPool1D(max_len-2)
dense = Dense(15)

# input_vector=TimeDistributed(embedding)(description_input)
# validation_vector=embedding(description_validation)

input_vector = TimeDistributed(embedding)(input_1)
validation_vector = embedding(input_2)

convolutional_vector = TimeDistributed(conv1d)(input_vector)
validation_conv = conv1d(validation_vector)

maxpooling_vector = TimeDistributed(maxpool1d)(convolutional_vector)
validation_maxpooling = maxpool1d(validation_conv)

middle_output = TimeDistributed(dense)(maxpooling_vector)
middle_validation = dense(validation_maxpooling)

In [10]:
print(input_vector.shape, validation_vector.shape,
      convolutional_vector.shape, validation_conv.shape,
      maxpooling_vector.shape, validation_maxpooling.shape,
      middle_output.shape, middle_validation.shape)

(None, 3, 1253, 50) (None, 1253, 50) (None, 3, 1251, 64) (None, 1251, 64) (None, 3, 1, 64) (None, 1, 64) (None, 3, 1, 15) (None, 1, 15)


In [11]:
def change_dim_1(X):
    return K.squeeze(X, 1)

def change_dim_2(X):
    return K.squeeze(X, 2)

def repeat(X):
    return K.repeat_elements(X, description_num, 1)

def repeat1(X):
    return K.repeat_elements(X, 15, 2)

def repeat2(X):
    return K.repeat_elements(X, description_num, 1)

def dot(X, Y):
    return K.dot(X, Y)

def sum_item(X):
    return K.sum(X, axis=2)

def sqrt_item(X):
    return K.sqrt(X)

def cal_denominator(X):
    return 1/(X)

def expand_item(X):
    return K.expand_dims(X, 2)

def expand_rate(X):
    return K.expand_dims(X, 1)

def sum_user(X):
    return K.sum(X, 1)

def sum_rate(X):
    return K.sum(X, 1)

def single_exp(X):
    return K.exp(X)

def sum_exp_denominator(X):
    return 1/(K.sum(K.exp(X), 1))

In [12]:
# def _test_generator(filename):

middle_output_final = Lambda(change_dim_2)(middle_output)#?*10*15
middle_validation_final = Lambda(repeat)(middle_validation)#?*10*15
molecule = Multiply()([middle_output_final, middle_validation_final])
molecule = Lambda(sum_item)(molecule)

denominator1 = Multiply()([middle_output_final, middle_output_final])
denominator1 = Lambda(sum_item)(denominator1)
denominator1 = Lambda(sqrt_item)(denominator1)

denominator2 = Multiply()([middle_validation_final, middle_validation_final])
denominator2 = Lambda(sum_item)(denominator2)
denominator2 = Lambda(sqrt_item)(denominator2)


denominator = Multiply()([denominator1, denominator2])
denominator = Lambda(cal_denominator)(denominator)

similarity = Multiply()([molecule, denominator])
similarity = Lambda(expand_item)(similarity)
similarity = Lambda(repeat1)(similarity)


user = Multiply()([similarity, middle_output_final])
user = Lambda(sum_user)(user)


item = Lambda(change_dim_1)(middle_validation)


rate_hat = Multiply()([user, item])
rate_hat = Lambda(sum_rate)(rate_hat)
rate_hat = Lambda(expand_rate)(rate_hat)

In [13]:
model = Model(inputs=[input_1, input_2], outputs=rate_hat)
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', 'mape'])

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3, 1253)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1253)]       0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 3, 1253, 50)  1573300    ['input_1[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 1253, 50)     1573300     ['input_2[0][0]']            

In [15]:
hist = model.fit(train_data, train_target,
                 validation_data=(valid_data, valid_target),
                 steps_per_epoch=256,
                 validation_steps=256,
                 epochs=30,
                 verbose=1)

Epoch 1/30
256/256 [==============================] - 60s 188ms/step - loss: 4.6927 - mse: 4.6927 - mae: 1.4941 - mape: 23.0244 - val_loss: 2.4495 - val_mse: 2.4495 - val_mae: 1.1641 - val_mape: 20.9025
Epoch 2/30
256/256 [==============================] - 48s 188ms/step - loss: 2.3059 - mse: 2.3059 - mae: 1.1598 - mape: 19.5883 - val_loss: 2.5248 - val_mse: 2.5248 - val_mae: 1.1400 - val_mape: 21.1270
Epoch 3/30
256/256 [==============================] - 49s 191ms/step - loss: 2.2859 - mse: 2.2859 - mae: 1.1524 - mape: 19.4893 - val_loss: 2.4079 - val_mse: 2.4079 - val_mae: 1.2012 - val_mape: 20.6837
Epoch 4/30
256/256 [==============================] - 49s 191ms/step - loss: 2.2715 - mse: 2.2715 - mae: 1.1479 - mape: 19.4107 - val_loss: 2.3938 - val_mse: 2.3938 - val_mae: 1.1561 - val_mape: 20.6722
Epoch 5/30
256/256 [==============================] - 49s 191ms/step - loss: 2.2553 - mse: 2.2553 - mae: 1.1433 - mape: 19.3394 - val_loss: 2.3778 - val_mse: 2.3778 - val_mae: 1.1751 - val

In [16]:
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_5s.txt','test_whole_list_5s.txt'),steps=134103)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_10.txt','test_whole_list_10.txt'),steps=34266)
# scores = model.evaluate_generator(_evaluate_generator('train_whole_list_6s.txt','test_whole_list_6s.txt'),steps=134103)
scores = model.evaluate(test_data, test_target, steps=2500)
print(scores)

2500/2500 [==============================] - 10s 4ms/step - loss: 2.2327 - mse: 2.2327 - mae: 1.1107 - mape: 19.0911
[2.2326736450195312, 2.232673406600952, 1.1107429265975952, 19.091140747070312]


In [ ]:
#model.save(path+"IMDB_ADLFM_baseline")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/캡스톤/data/IMDB_ADLFM_baseline/assets
